<a href="https://colab.research.google.com/github/adityasingh1993/100DayMLChallenge/blob/master/Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!wget http://www.manythings.org/anki/deu-eng.zip

--2018-11-22 16:46:05--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4162728 (4.0M) [application/zip]
Saving to: ‘deu-eng.zip’

deu-eng.zip         100%[===================>]   3.97M  --.-KB/s    in 0.1s    

2018-11-22 16:46:05 (32.5 MB/s) - ‘deu-eng.zip’ saved [4162728/4162728]



In [7]:
!ls

deu-eng.zip  sample_data


In [8]:
!unzip deu-eng.zip

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              


In [0]:
import re
import string
from pickle import dump
from unicodedata import normalize
import numpy as np
import array


In [0]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

In [0]:

from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

In [0]:
def load_doc(filename):
  file=open(filename,mode='rt',encoding='utf-8')
  text=file.read()
  file.close()
  return text

In [0]:
def to_pairs(doc):
  lines=doc.strip().split('\n')
  pairs=[line.split('\t') for line in lines ]
  return pairs

In [0]:
def clean_pairs(lines):
  cleaned=[]
  print(len(lines))
  re_print=re.compile('^[%s]'% re.escape(string.printable))
  table=str.maketrans('','',string.punctuation)
  for pair in lines:
   
    clean_pair=[]
    for line in pair:
      line=normalize('NFD',line).encode('ascii','ignore')
     
      line=line.decode('UTF-8')
      
      line=line.split()
      
      line=[word.lower() for word in line]
      line=[word.translate(table) for word in line]
      line=[re_print.sub('',w) for w in line]
      line=[word for word in line if word.isalpha() ]
      clean_pair.append(' '.join(line))
    cleaned.append(clean_pair)
  return cleaned

In [0]:
def save_clean_Data(sentences,filename):
  dump(sentences,open(filename,'wb'))
  print("Saved {0}".format(filename))
  

In [0]:
filename='deu.txt'

In [0]:
doc=load_doc(filename)

In [0]:
pairs=to_pairs(doc)

In [58]:
cleanpairs=clean_pairs(pairs)

176692


In [0]:
cleanpairs=np.asarray(cleanpairs)

In [60]:
save_clean_Data(cleanpairs, 'english-german.pkl')

Saved english-german.pkl


In [0]:
def load_clean_sentences(filename):
  return load(open(filename,'rb'))

In [0]:
def save_clean_data(sentences,filename):
  dump(sentence,open(filename,'wb'))
  print("File saved {0}".format(filename))

In [0]:
raw_dataset=load_clean_sentences('english-german.pkl')

In [0]:
n_sentences=10000
dataset=raw_dataset[:n_sentences,:]
shuffle(dataset)

In [0]:
train,test=dataset[:9000],dataset[9000:]

In [66]:
save_clean_Data(dataset,'english-german-both.pkl')
save_clean_Data(train,'english-german-train.pkl')
save_clean_Data(test,'english-german-test.pkl')

Saved english-german-both.pkl
Saved english-german-train.pkl
Saved english-german-test.pkl


In [0]:
def create_tokenizer(lines):
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [0]:
def max_len(lines):
  return max(len(line.split())for line in lines)

In [0]:
eng_tokenizer=create_tokenizer(dataset[:,0])

In [0]:
eng_vocab_size=len(eng_tokenizer.word_index)+1

In [0]:
eng_lenth=max_len(dataset[:,0])

In [0]:
ger_tokenizer=create_tokenizer(dataset[:,1])

In [0]:
ger_vocab_size=len(ger_tokenizer.word_index)+1

In [0]:
ger_lenth=max_len(dataset[:,1])

In [0]:
def encode_sequences(tokenizer,length,lines):
  X=tokenizer.texts_to_sequences(lines)
  X=pad_sequences(X,maxlen=length,padding='post')
  return X

In [0]:
def encode_output(sequences,vocab_size):
  ylist=[]
  for sequence in sequences:
    encoded=to_categorical(sequence,num_classes=vocab_size)
    ylist.append(encoded)
  y=np.asarray(ylist)
  y=y.reshape(sequences.shape[0],sequences.shape[1],vocab_size)
  return y

In [0]:
trainX=encode_sequences(ger_tokenizer,ger_lenth,train[:,1])
trainY=encode_sequences(eng_tokenizer,eng_lenth,train[:,0])





In [0]:
trainY=encode_output(trainY,eng_vocab_size)

In [0]:
testX=encode_sequences(ger_tokenizer,ger_lenth,test[:,1])
testY=encode_sequences(eng_tokenizer,eng_lenth,test[:,0])
testY=encode_output(testY,eng_vocab_size)

In [0]:
def define_Model(src_vocab,tar_vocab,src_timestamps,tar_timestamps,n_units):
  model=Sequential()
  model.add(Embedding(src_vocab,n_units,input_length=src_timestamps,mask_zero=True))
  model.add(LSTM(n_units))
  model.add(RepeatVector(tar_timestamps))
  model.add(LSTM(n_units,return_sequences=True))
  model.add(TimeDistributed(Dense(tar_vocab,activation='softmax')))
  return model

In [0]:
model=define_Model(ger_vocab_size,eng_vocab_size,ger_lenth,eng_lenth,256)
model.compile(optimizer='adam',loss='categorical_crossentropy')


In [82]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 256)           830976    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 1950)           501150    
Total params: 2,382,750
Trainable params: 2,382,750
Non-trainable params: 0
_________________________________________________________________
None


In [83]:
filename="model.h5"
checkpoint=ModelCheckpoint(filename,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
model.fit(trainX,trainY,epochs=30,batch_size=64)

Epoch 1/30
9000/9000 [==============================] - 35s 4ms/step - loss: 3.9053
Epoch 2/30
9000/9000 [==============================] - 32s 4ms/step - loss: 3.0847
Epoch 3/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.9842
Epoch 4/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.9144
Epoch 5/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.8434
Epoch 6/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.7044
Epoch 7/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.5653
Epoch 8/30
9000/9000 [==============================] - 33s 4ms/step - loss: 2.4420
Epoch 9/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.3081
Epoch 10/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.1844
Epoch 11/30
9000/9000 [==============================] - 32s 4ms/step - loss: 2.0635
Epoch 12/30
9000/9000 [==============================] - 32s 4ms/step - lo

In [0]:
def word_for_id(integer,tokenizer):
  for word,index in tokenizer.word_index.items():
    if index==integer:
      return word
  return None

In [0]:
def predict_sequenc(model,tokenizer,source):
  prediction=model.predict(source,verbose=0)[0]
  integer=[argmax(vector) for vector in prediction]
  target=list()
  for i in integer:
    word=word_for_id(i, tokenizer)
    if word is None:
      break
    target.append(word)
  return ' '.join(target)

In [0]:
def evaluate_model(model,tokenizer,sources,raw_dataset):
  actual,predicted=[],[]
  for i,source in enumerate(sources):
    source=source.reshape((1,source.shape[0]))
    transalation=predict_sequence(model,tokenizer,source)
    predicted.append(translation)
  return predicted

In [0]:
evaluate_model(model, eng_tokenizer, testX, test)